# Week 6: Dynamics on Networks

**Learning objectives** — After this lab you should be able to:

- Explain the SIR model compartments and parameters (beta, gamma, R_0)
- Simulate the mean-field SIR ODE and interpret the curves
- Run stochastic SIR on a network and compare to the mean-field
- Visualize epidemic snapshots on a graph
- Compare hub seeding vs random seeding for information cascades
- Explain how community structure affects cascade spread
- Compare random, targeted, and acquaintance immunization strategies
- Simulate the voter model and explain how network structure affects consensus time

So far we have studied the **structure** of networks. This week we study what happens
**on** networks — how diseases, information, and influence spread through connections.

The key insight: network structure profoundly affects dynamics.
Hubs accelerate epidemics, and targeted immunization of hubs can stop them.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.integrate import odeint
from netsci.loaders import load_graph
from netsci.utils import SEED, graph_summary
from netsci.dynamics import (
    network_sir,
    independent_cascade,
    immunize_and_simulate,
    acquaintance_immunize,
    voter_model,
)

---
## 1. Datasets

We use two real networks this week:

- **US Airports** (500 nodes, ~2,980 edges) — a hub-and-spoke topology where a few major airports connect to hundreds of others. This structure accelerates epidemic spread because infections can jump between distant regions in a single hop through a hub.
- **Facebook** (334 nodes, ~2,852 edges) — a dense social network with strong community structure. Tight friend-groups act as local "firewalls" that can slow cascades but also trap infections within clusters.

In [ ]:
G_air = load_graph("airports")
graph_summary(G_air)
print()
G_fb = load_graph("facebook")
graph_summary(G_fb)

---
## 2. SIR Model Intuition

The **SIR model** divides a population into three compartments:

- **S** (Susceptible) — can catch the disease
- **I** (Infected) — currently sick and can spread the disease
- **R** (Recovered) — immune (or removed)

The flow is: **S → I → R** (no going back!).

Two key parameters:
- **beta** (β): infection rate — probability of transmission per contact
- **gamma** (γ): recovery rate — probability of recovering per time step

The **basic reproduction number** R₀ = β/γ tells us whether the epidemic grows (R₀ > 1) or dies out (R₀ < 1).

In [ ]:
# S → I → R compartment diagram
with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(8, 2.5))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 3)
    ax.axis("off")

    boxes = [("S", 1.5, "#4878CF"), ("I", 5.0, "#D65F5F"), ("R", 8.5, "#8C8C8C")]
    for label, x, color in boxes:
        rect = mpatches.FancyBboxPatch(
            (x - 0.7, 0.8),
            1.4,
            1.4,
            boxstyle="round,pad=0.15",
            fc=color,
            ec="black",
            lw=1.5,
        )
        ax.add_patch(rect)
        ax.text(
            x,
            1.5,
            label,
            ha="center",
            va="center",
            fontsize=22,
            fontweight="bold",
            color="white",
        )

    ax.annotate(
        "",
        xy=(3.6, 1.5),
        xytext=(2.5, 1.5),
        arrowprops=dict(arrowstyle="-|>", lw=2, color="black"),
    )
    ax.text(3.05, 2.0, r"$\beta \cdot S \cdot I$", ha="center", fontsize=12)

    ax.annotate(
        "",
        xy=(7.1, 1.5),
        xytext=(6.0, 1.5),
        arrowprops=dict(arrowstyle="-|>", lw=2, color="black"),
    )
    ax.text(6.55, 2.0, r"$\gamma \cdot I$", ha="center", fontsize=12)

    fig.suptitle("SIR Compartment Flow", fontsize=14, y=0.98)
    fig.tight_layout()
    plt.show()

---
## 3. Mean-Field SIR (ODE)

The simplest version assumes everyone mixes uniformly (no network structure).

The three ODE equations translate directly into code:

- `dS/dt = -β·S·I` — the susceptible pool shrinks in proportion to both S and I (more contacts = more infections)
- `dI/dt = +β·S·I - γ·I` — the infected pool gains from new infections and loses from recoveries
- `dR/dt = +γ·I` — the recovered pool grows as infected individuals heal

Notice: S + I + R = 1 at all times (the population is conserved).

In [ ]:
def sir_ode(y, t, beta, gamma):
    """SIR ordinary differential equations."""
    S, I, R = y
    dSdt = -beta * S * I
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I
    return [dSdt, dIdt, dRdt]


# Parameters
beta_ode, gamma_ode = 0.3, 0.1
R0 = beta_ode / gamma_ode
print(f"R_0 = beta/gamma = {R0:.1f}")

# Initial conditions: 99.9% susceptible, 0.1% infected
y0 = [0.999, 0.001, 0.0]
t = np.linspace(0, 160, 1000)

solution = odeint(sir_ode, y0, t, args=(beta_ode, gamma_ode))
S, I, R = solution.T

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(t, S, label="Susceptible", color="#4878CF")
    ax.plot(t, I, label="Infected", color="#D65F5F")
    ax.plot(t, R, label="Recovered", color="#8C8C8C")
    ax.set_xlabel("Time")
    ax.set_ylabel("Fraction of population")
    ax.set_title(f"Mean-Field SIR (R₀ = {R0:.1f})")
    ax.legend()
    fig.tight_layout()
    plt.show()

**Reading the curves**: S decreases monotonically — people only leave the susceptible pool. I peaks then falls as recovery outpaces infection. R increases monotonically and represents the cumulative epidemic size. The peak of I is the moment of maximum strain on healthcare — "flattening the curve" means reducing this peak by lowering R₀. With R₀ = 3.0, roughly 94% of the population eventually gets infected.

**Before you tweak**: What happens when R₀ drops below 1 (try β = 0.05, γ = 0.1)? Predict whether the I curve will still peak, then run the tweak cell to check.

**Try it yourself**: What is R₀ for beta=0.3, gamma=0.1? Will the epidemic grow or die out? What about beta=0.05, gamma=0.1?

In [ ]:
# YOUR CODE HERE
R0_a = 0.3 / 0.1  # beta=0.3, gamma=0.1
R0_b = 0.05 / 0.1  # beta=0.05, gamma=0.1

assert R0_a == 0.3 / 0.1, "Hint: R0 = beta / gamma"
assert R0_b == 0.05 / 0.1, "Hint: R0 = beta / gamma"
print(
    f"Case A: R0 = {R0_a:.1f} -> {'Epidemic GROWS (R0 > 1)' if R0_a > 1 else 'Epidemic DIES OUT (R0 < 1)'}"
)
print(
    f"Case B: R0 = {R0_b:.1f} -> {'Epidemic GROWS (R0 > 1)' if R0_b > 1 else 'Epidemic DIES OUT (R0 < 1)'}"
)

In [ ]:
# ---- TWEAK: Change beta and gamma ----
beta_tw = 0.3  # <-- change me (try 0.1, 0.3, 0.5)
gamma_tw = 0.1  # <-- change me (try 0.05, 0.1, 0.2)

R0_tw = beta_tw / gamma_tw
solution_tw = odeint(sir_ode, y0, t, args=(beta_tw, gamma_tw))
S_tw, I_tw, R_tw = solution_tw.T

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(t, S_tw, label="S", color="#4878CF")
    ax.plot(t, I_tw, label="I", color="#D65F5F")
    ax.plot(t, R_tw, label="R", color="#8C8C8C")
    ax.set_xlabel("Time")
    ax.set_ylabel("Fraction")
    ax.set_title(f"SIR ODE: β={beta_tw}, γ={gamma_tw}, R₀={R0_tw:.1f}")
    ax.legend()
    fig.tight_layout()
    plt.show()

---
## 4. Stochastic SIR on a Network

Real people don't mix uniformly — they have specific contacts.
Let's simulate SIR **on the airport network**, where infection can only spread along edges.

The `network_sir()` function below implements stochastic SIR on a graph. The algorithm has three phases each time step:

1. **Infection step** — each infected node independently "rolls the dice" for each susceptible neighbor (probability β). This is where network structure matters: hubs attempt many more infections per step.
2. **Recovery step** — each infected node recovers with probability γ, independently of neighbors.
3. **Update** — newly infected join I, newly recovered move to R. All updates happen simultaneously (no node acts on changes from the same step).

The `rng` parameter ensures reproducible randomness across runs.

The `network_sir()` function is now available from `netsci.dynamics`. It implements the stochastic SIR algorithm described above: at each time step, infected nodes attempt to infect susceptible neighbors (probability beta), then each infected node recovers independently (probability gamma).

In [ ]:
# Single run on airports
result = network_sir(G_air, beta=0.05, gamma=0.1, n_seeds=3)
N = G_air.number_of_nodes()

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(8, 5))
    steps = range(len(result["S"]))
    ax.plot(steps, [s / N for s in result["S"]], label="S", color="#4878CF")
    ax.plot(steps, [i / N for i in result["I"]], label="I", color="#D65F5F")
    ax.plot(steps, [r / N for r in result["R"]], label="R", color="#8C8C8C")
    ax.set_xlabel("Time step")
    ax.set_ylabel("Fraction")
    ax.set_title("Network SIR on Airports (single run)")
    ax.legend()
    fig.tight_layout()
    plt.show()

**Compare to the ODE**: Notice how jagged the network SIR curve is compared to the smooth ODE. Each run is different because infection is stochastic — some runs barely take off (the seeds happen to be low-degree nodes), while others explode (a hub gets infected early). The ODE is the "average over infinitely many runs on a fully connected population."

**Why simulate stochastically?** The ODE model assumes every node has the same number of neighbors (homogeneous mixing). Real networks have hubs and communities that create uneven spreading. Stochastic simulation on the actual network graph captures this heterogeneity — a hub airport getting infected early can cause a dramatically different epidemic trajectory than a peripheral airport.

In [ ]:
# Monte Carlo: 20 runs, plot mean +/- std
n_runs = 20
max_steps = 200
beta_net, gamma_net = 0.05, 0.1
rng = np.random.default_rng(SEED)

all_I = []
for run in range(n_runs):
    res = network_sir(
        G_air, beta_net, gamma_net, n_seeds=3, max_steps=max_steps, rng=rng
    )
    # Pad to max_steps + 1 if epidemic died early
    I_curve = res["I"] + [0] * (max_steps + 1 - len(res["I"]))
    all_I.append(I_curve[: max_steps + 1])

all_I = np.array(all_I) / N
mean_I = all_I.mean(axis=0)
std_I = all_I.std(axis=0)

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(8, 5))
    steps = range(max_steps + 1)
    ax.plot(steps, mean_I, color="#D65F5F", label="Mean infected")
    ax.fill_between(
        steps,
        mean_I - std_I,
        mean_I + std_I,
        color="#D65F5F",
        alpha=0.2,
        label="± 1 std",
    )
    ax.set_xlabel("Time step")
    ax.set_ylabel("Fraction infected")
    ax.set_title(f"Network SIR Monte Carlo ({n_runs} runs)")
    ax.legend()
    fig.tight_layout()
    plt.show()

**Reading the band**: The shaded ±1 std region is widest near the epidemic peak — this is when the outcome is most uncertain. Early on, there are few infected (low variance). Late, most runs have converged (nearly everyone recovered). The peak timing and height are the most variable quantities across runs.

---
## 5. Epidemic Snapshots

Let's visualize the infection spreading through the network at different time points.

In the snapshots below, watch for:
- **Color wavefront**: green (S) → red (I) → gray (R) spreading outward from the initial seeds
- **Hub effects**: high-degree airports turn red early and broadcast infection to many neighbors simultaneously
- **Geographic clusters**: some regions may stay green longer if they are weakly connected to the initial outbreak

In [ ]:
# Run a single epidemic for snapshots
result_snap = network_sir(
    G_air, beta=0.05, gamma=0.1, n_seeds=3, rng=np.random.default_rng(SEED)
)
T = len(result_snap["states"]) - 1
snapshot_times = [0, T // 4, T // 2, T]

# Use a fixed layout for consistency
pos = nx.spring_layout(G_air, seed=SEED)
state_colors = {"S": "#6ACC65", "I": "#D65F5F", "R": "#8C8C8C"}

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for ax, t_idx in zip(axes, snapshot_times):
    t_idx = min(t_idx, len(result_snap["states"]) - 1)
    states = result_snap["states"][t_idx]
    colors = [state_colors[states[n]] for n in G_air.nodes()]
    nx.draw_networkx(
        G_air,
        pos,
        ax=ax,
        node_color=colors,
        node_size=15,
        edge_color="#cccccc",
        width=0.2,
        with_labels=False,
        alpha=0.8,
    )
    n_s = sum(1 for v in states.values() if v == "S")
    n_i = sum(1 for v in states.values() if v == "I")
    n_r = sum(1 for v in states.values() if v == "R")
    ax.set_title(f"t = {t_idx}\nS={n_s} I={n_i} R={n_r}")
    ax.axis("off")

fig.suptitle("Epidemic spreading on airport network", fontsize=14)
fig.tight_layout()
plt.show()

**What you see**: The infection radiates outward from the seed nodes. By t = T/4, hubs are typically already recovered (gray), having served as "superspreader" relay points. The final snapshot shows expanding shells of gray (recovered) surrounding any remaining green (susceptible) pockets — these are the nodes that the epidemic never reached, often in peripheral parts of the network.

---
## 6. Information Cascades

Similar to epidemics: a piece of information starts at seed nodes and spreads
to neighbors with some probability. Let's compare **hub seeds** vs **random seeds**.

An **independent cascade** is essentially "SIR without recovery" — once a node is activated, it stays active forever and gets exactly one chance to activate each neighbor (with probability p). This models information spread: once you've heard a rumor, you might tell your friends, but you only try once.

The key question: does seeding hubs versus random nodes make a difference?

In [ ]:
# Compare hub seeds vs random seeds on Facebook
rng = np.random.default_rng(SEED)
n_trials = 30
n_seeds = 3

# Hub seeds: top-3 by degree
hubs = sorted(G_fb.nodes(), key=lambda n: G_fb.degree(n), reverse=True)[:n_seeds]
hub_results = []
for _ in range(n_trials):
    activated = independent_cascade(G_fb, hubs, p=0.1, rng=rng)
    hub_results.append(len(activated) / G_fb.number_of_nodes())

# Random seeds
random_results = []
for _ in range(n_trials):
    rand_seeds = list(rng.choice(list(G_fb.nodes()), size=n_seeds, replace=False))
    activated = independent_cascade(G_fb, rand_seeds, p=0.1, rng=rng)
    random_results.append(len(activated) / G_fb.number_of_nodes())

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.boxplot([hub_results, random_results], labels=["Hub seeds", "Random seeds"])
    ax.set_ylabel("Fraction activated")
    ax.set_title("Information Cascade: Hub vs Random Seeding")
    fig.tight_layout()
    plt.show()

print(f"Hub seeds: {np.mean(hub_results):.2%} ± {np.std(hub_results):.2%}")
print(f"Random seeds: {np.mean(random_results):.2%} ± {np.std(random_results):.2%}")

**Hub seeds vs random seeds**: Seeding an epidemic at a hub node reaches more nodes faster because the hub's many connections act as simultaneous transmission channels. This is why superspreader events matter — a single well-connected individual at a large gathering can ignite an outbreak that dozens of isolated infections could not.

**Hub advantage**: Seeding the top-3 hubs reaches significantly more of the network than random seeds. This is because hubs have direct access to many nodes in the first hop, and their neighbors are often well-connected themselves. Random seeds may land on peripheral nodes with degree 2-3, limiting the cascade's initial reach.

**Connection to communities (Week 5)**: Cascades spread quickly *within* communities (dense internal connections) but struggle to jump *between* communities (sparse bridge edges). A seed inside a tight community may saturate that community but fail to escape. Hub seeds are effective precisely because hubs tend to bridge multiple communities, giving the cascade simultaneous footholds in different parts of the network.

---
## 7. Immunization Strategies

If we can vaccinate some fraction of the population, **which nodes should we choose?**

In Week 4 we saw that targeted hub removal shatters scale-free networks. Now let's use
this insight for epidemic control.

- **Random immunization**: remove nodes at random
- **Targeted immunization**: remove the highest-degree nodes (hubs) first

The key insight: targeted immunization is far more effective on networks with hubs.

Two immunization strategies represent opposite ends of a cost-information tradeoff:

- **Random immunization** requires no knowledge of the network — just vaccinate people at random. Simple and fair, but inefficient on networks with hubs.
- **Targeted immunization** removes the highest-degree nodes first. Extremely effective but requires knowing the full degree sequence, which is rarely available in practice.

In [ ]:
# Sweep immunization fraction
fractions = np.arange(0, 0.31, 0.05)
peaks_random = [immunize_and_simulate(G_air, f, "random") for f in fractions]
peaks_targeted = [immunize_and_simulate(G_air, f, "targeted") for f in fractions]

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.plot(fractions * 100, peaks_random, "o-", label="Random", markersize=6)
    ax.plot(
        fractions * 100, peaks_targeted, "s-", label="Targeted (hubs)", markersize=6
    )
    ax.set_xlabel("% of nodes immunized")
    ax.set_ylabel("Mean peak infected (fraction)")
    ax.set_title("Immunization Strategies on Airport Network")
    ax.legend()
    fig.tight_layout()
    plt.show()

**Key observation**: removing just 10-15% of the highest-degree nodes dramatically reduces the epidemic peak. Random removal of the same fraction barely helps. This is because hubs are the "superspreaders" that connect different parts of the network.

### Acquaintance Immunization

We saw that targeted immunization requires knowing the full degree sequence — rarely available in practice. **Acquaintance immunization** offers a clever middle ground:

1. Pick a random person
2. Ask them to name a friend
3. Vaccinate the friend (not the original person)

By the **friendship paradox**, the named friend has higher-than-average degree. This biased sampling preferentially hits hubs *without needing global network information*.

In [ ]:
# Compare all three strategies
fractions_imm = np.arange(0, 0.31, 0.05)
peaks_random_3 = []
peaks_targeted_3 = []
peaks_acquaintance = []

for f in fractions_imm:
    rng_acq = np.random.default_rng(SEED)
    if f == 0.0:
        res = immunize_and_simulate(G_air, 0, "none")
        peaks_random_3.append(res)
        peaks_targeted_3.append(res)
        peaks_acquaintance.append(res)
    else:
        peaks_random_3.append(immunize_and_simulate(G_air, f, "random"))
        peaks_targeted_3.append(immunize_and_simulate(G_air, f, "targeted"))
        # Acquaintance
        G_acq = acquaintance_immunize(G_air, f, rng_acq)
        if G_acq.number_of_nodes() > 4:
            rng_sim = np.random.default_rng(SEED)
            acq_peaks = []
            for _ in range(20):
                res_acq = network_sir(
                    G_acq, 0.05, 0.1, n_seeds=3, max_steps=100, rng=rng_sim
                )
                acq_peaks.append(max(res_acq["I"]) / G_air.number_of_nodes())
            peaks_acquaintance.append(np.mean(acq_peaks))
        else:
            peaks_acquaintance.append(0.0)

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.plot(fractions_imm * 100, peaks_random_3, "o-", label="Random", markersize=6)
    ax.plot(
        fractions_imm * 100,
        peaks_acquaintance,
        "^-",
        label="Acquaintance",
        markersize=6,
    )
    ax.plot(
        fractions_imm * 100,
        peaks_targeted_3,
        "s-",
        label="Targeted (hubs)",
        markersize=6,
    )
    ax.set_xlabel("% of nodes immunized")
    ax.set_ylabel("Mean peak infected (fraction)")
    ax.set_title("Three Immunization Strategies on Airport Network")
    ax.legend()
    fig.tight_layout()
    plt.show()

**Acquaintance immunization in action**: The acquaintance strategy performs significantly better than random immunization and approaches targeted immunization — all without needing global knowledge of the network. This is the friendship paradox at work: by asking random people to name friends, we oversample high-degree nodes (hubs), achieving near-targeted effectiveness with only local information.

This has real public health implications: during an outbreak, you don't need a census of everyone's contacts. Just ask random people "who do you interact with most?" and prioritize vaccinating those named individuals.

---
## 8. Opinion Dynamics: The Voter Model

Epidemics are one type of dynamics on networks. Another fundamental process is **opinion formation** — how do competing opinions spread and when does a population reach consensus?

The **voter model** is the simplest model of opinion dynamics:
1. Each node holds one of two opinions (say, 0 or 1)
2. At each time step, pick a random node
3. That node copies the opinion of a randomly chosen neighbor
4. Repeat until consensus (all nodes agree) or a time limit

Unlike SIR (which has a one-way flow S→I→R), the voter model allows opinions to flip back and forth. The key question: **how does network structure affect the time to reach consensus?**

In [ ]:
# Run voter model on Facebook network
rng_voter = np.random.default_rng(SEED)
result_voter = voter_model(G_fb, max_steps=50000, rng=rng_voter)

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(8, 4))
    steps = np.arange(len(result_voter["fraction_1"])) * 50
    ax.plot(steps, result_voter["fraction_1"], color="#4878CF", linewidth=1.0)
    ax.axhline(0.5, color="#999999", linestyle="--", alpha=0.5, label="50/50 split")
    ax.set_xlabel("Time step")
    ax.set_ylabel("Fraction with opinion 1")
    ax.set_title("Voter Model on Facebook Network (single run)")
    ax.set_ylim(-0.05, 1.05)
    ax.legend()
    fig.tight_layout()
    plt.show()

**Reading the voter model trajectory**: The fraction of opinion 1 follows a random walk that eventually absorbs at 0 or 1 (consensus). The trajectory is noisy because the model is inherently stochastic — at each step, a single node flips. On a well-connected network like Facebook, the walk can hover near 50/50 for a long time before one opinion wins by chance. The community structure can create temporary "plateaus" where each community locks into a different opinion, slowing consensus.

In [ ]:
# Compare voter model on different network structures
# Complete graph (well-mixed) vs ring lattice (slow mixing) vs Facebook (communities)
G_complete = nx.complete_graph(G_fb.number_of_nodes())
G_ring = nx.watts_strogatz_graph(G_fb.number_of_nodes(), 4, 0, seed=SEED)

n_runs = 10
max_steps = 30000

fig, axes = plt.subplots(1, 3, figsize=(16, 4))
networks = [
    ("Complete graph\n(well-mixed)", G_complete),
    ("Ring lattice\n(slow mixing)", G_ring),
    ("Facebook\n(communities)", G_fb),
]

for ax, (name, G) in zip(axes, networks):
    rng_v = np.random.default_rng(SEED)
    for run in range(n_runs):
        res = voter_model(G, max_steps=max_steps, rng=rng_v)
        steps_v = np.arange(len(res["fraction_1"])) * 50
        ax.plot(steps_v, res["fraction_1"], alpha=0.4, linewidth=0.8)
    ax.axhline(0.5, color="#999999", linestyle="--", alpha=0.3)
    ax.set_xlabel("Time step")
    ax.set_ylabel("Fraction opinion 1")
    ax.set_title(name, fontsize=11)
    ax.set_ylim(-0.05, 1.05)

fig.suptitle(
    "Voter Model: Network Structure Affects Consensus Speed",
    fontsize=13,
    fontweight="bold",
)
fig.tight_layout()
plt.show()

**Structure shapes consensus**: On a complete graph (left), every node is a neighbor of every other — opinions mix rapidly and consensus arrives quickly. On a ring lattice (center), information can only propagate locally, so it takes much longer for one opinion to dominate. Facebook (right) sits between these extremes: communities can reach internal consensus quickly, but the sparse between-community bridges slow global agreement. This echoes a real-world phenomenon — echo chambers form when communities internally agree but resist external influence.

**Epidemics vs opinions**: Notice a key difference from SIR. In epidemics, the process has a direction (S→I→R) and eventually dies out. In the voter model, opinions can flip indefinitely — the only absorbing states are full consensus. This makes the voter model a better analogy for political polarization, technology adoption, and cultural norms, where "recovery" (changing your mind) is always possible.

---
## Summary

| Concept | Key insight |
|---------|-------------|
| **SIR model** | S → I → R with rates β (infection) and γ (recovery) |
| **R₀ = β/γ** | Epidemic threshold — grows if R₀ > 1 |
| **Network SIR** | Structure matters — hubs accelerate spread |
| **Targeted immunization** | Removing hubs is far more effective than random |
| **Acquaintance immunization** | Friendship paradox enables near-targeted results with only local info |
| **Information cascades** | Hub seeds reach more people; communities slow cross-group spread |
| **Voter model** | Opinion dynamics where nodes copy neighbors; network structure affects consensus time |
| **Echo chambers** | Communities reach internal consensus quickly but resist cross-boundary influence |

See Week 4 for the Molloy-Reed criterion and robustness paradox.

This concludes the 6-week Network Science Lab Course. You now have the tools to:
- Build and analyze networks
- Measure their properties
- Detect communities
- Simulate dynamics (epidemics, opinions, cascades)
- Make informed decisions about network interventions